# Data Cleaning Day Files From FirstRate Data
It would also be nice to have day data, and to make sure we can get long enough sequences, we'll keep the day data all the way back to 6/1/20, as that's when the market started recovering from the COVID-19 crash.

**Update**: Change of plans, since there seems to be missing data, and it would take unnecessary time to obtain/clean that data, we'll just stick to getting the day data since 10/1/20.

## Imports

In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import heatmap
import dask
import dask.dataframe as dd
from dask import delayed
from pyarrow.parquet import ParquetFile
import pyarrow as pa
from tqdm import tqdm

import tulipy as ti

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import IncrementalPCA

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import initializers
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.optimizers import SGD
import keras_tuner as kt
from keras_tuner import HyperParameters

import os
import sys
import warnings
import shutil

## Getting the list of tickers that we need 1-day data for
We have our 2,599 tickers that has complete, continuous, and cleaned 1-min data. Now let's make sure that data exists for the day data as well.

In [7]:
# Getting the paths can making sure the lengths seem appropriate (day data should have more since it's not filtered)
cleaned_1min_parq_path = '/Volumes/T7/Filtered-Cleaned-Parquet-ML'
stocks_1day_path = '/Volumes/T7/stocks1day-csv'

print("Number of 1-min filtered and cleaned tickers:", len(os.listdir(cleaned_1min_parq_path)))
print("Number of 1-day unfiltered tickers total:", len(os.listdir(stocks_1day_path)))

Number of 1-min filtered and cleaned tickers: 2599
Number of 1-day unfiltered tickers total: 7592


In [12]:
# Getting the list of the tickers in the cleaned_1min_parq_path folder:
ticker_list_filtered = []

for folder_name in os.listdir(cleaned_1min_parq_path):
    ticker = folder_name.split('_')[0]
    ticker_list_filtered.append(ticker)

len(ticker_list_filtered)

2599

## Checking to see if the FRD day data is worth using and cleaning
We want to see if the FRD 1-day data is appropriate for us to use. If it's too unclean, we're better off creating our own OHLCV values from the 1-min FRD data.

In [49]:
# Getting the total number of trading days a stock should have from 6/1/20-9/30/20 (using AAPL b/c it definitely has all the days)
AAPL_path = '/Volumes/T7/stocks1day-csv/AAPL_full_1day_UNADJUSTED.csv'
AAPL_df = pd.read_csv(AAPL_path)
AAPL_df['Datetime'] = pd.to_datetime(AAPL_df['Datetime'])

start_date = pd.to_datetime("2020-06-01")
end_date = pd.to_datetime("2020-10-01")
num_days_2020 = len(AAPL_df[(AAPL_df['Datetime'] >= start_date) & (AAPL_df['Datetime'] < end_date)])
print("Number of days each relevant ticker should have in 2020:", num_days_2020)

Number of days each relevant ticker should have in 2020: 86


In [50]:
# Getting list of tickers without all the day data
missing_days_tickers = []

# Making sure that each ticker in the 1-min folder 1) exists in the day folder and 2) has 86 days of data from 6/1/20-9/30/20
for ticker in tqdm(ticker_list_filtered):

    # Getting the ticker file name and path
    ticker_1day_file = ticker + "_full_1day_UNADJUSTED.csv"
    ticker_1day_path = os.path.join(stocks_1day_path, ticker_1day_file)

    # Reading in the csv and converting to Datetime column to proper datetime format
    df = pd.read_csv(ticker_1day_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Checking to see if the number of days is not equal to the expected number (86 days from 6/1/20-9/30/20)
    num_days_ticker = len(df[(df['Datetime'] >= start_date) & (df['Datetime'] < end_date)])    
    if num_days_ticker != num_days_2020:
        print(ticker, "only has", num_days_ticker, "days of day data.")
        missing_days_tickers.append(ticker)


  1%|▌                                       | 34/2599 [00:00<00:14, 171.04it/s]

ASLE only has 67 days of day data.
ASAN only has 0 days of day data.
ACCD only has 61 days of day data.


  4%|█▍                                      | 93/2599 [00:00<00:13, 189.93it/s]

AOUT only has 26 days of day data.
API only has 65 days of day data.
ANNX only has 46 days of day data.
ALIT only has 51 days of day data.


  6%|██▎                                    | 152/2599 [00:00<00:13, 182.00it/s]

...

 93%|███████████████████████████████████▍  | 2427/2599 [00:12<00:00, 190.55it/s]

HPK only has 22 days of day data.
MAXN only has 22 days of day data.


 98%|█████████████████████████████████████▎| 2555/2599 [00:13<00:00, 206.08it/s]

ACI only has 65 days of day data.
SII only has 66 days of day data.


100%|██████████████████████████████████████| 2599/2599 [00:13<00:00, 194.69it/s]


In [52]:
# Checking out how many tickers are missing day data since 6/1/20
len(missing_days_tickers)

106

Indeed, we see that even requiring just day data from 6/1/20 to 9/30/20, we're missing 106 days of tickers. Furthermore, we're unsure as to how clean that data really is. Instead, we'll get the day data ourselves from the FRD original 1-min data.

## Checking the original FRD 1-min data to see if there's enough days of past data
Before we use the original FRD 1-min data, let's check if we have the appropriate number of days for the 1-min data. If there isn't, we might have to pivot and not use those extra days.

In [66]:
# Getting the AAPL path to check for appropriate number of days
AAPL_path = '/Volumes/T7/Original_FRD_Data_Till_Dec_2024/unzipped-FRD-csv-full/AAPL_full_1min_UNADJUSTED.csv'
AAPL_df = pd.read_csv(AAPL_path)
AAPL_df['Datetime'] = pd.to_datetime(AAPL_df['Datetime'])
start_date = pd.to_datetime("2020-06-01")
end_date = pd.to_datetime("2020-10-01")

# Getting the number of days from 6/1/20-9/30/20 and from 10/1/20 to the end of the data
num_days_extra = AAPL_df[(AAPL_df['Datetime'] >= start_date) & (AAPL_df['Datetime'] < end_date)]['Datetime'].dt.date.nunique()
num_days_orig = AAPL_df[(AAPL_df['Datetime'] >= end_date)]['Datetime'].dt.date.nunique()
print("Number of days of data from 6/1/20 to 9/30/20:", num_days_extra)
print("Number of days of data since 10/1/20 (original):", num_days_orig)

Number of days of data from 6/1/20 to 9/30/20: 86
Number of days of data since 10/1/20 (original): 1055


Firstly, we confirm that we still have 1,055 days (as expected), and we also confirm again that we have 86 days from 6/1/20-9/30/20.

In [67]:
# Setting some initial paths and variables
root_path_orig = '/Volumes/T7/Original_FRD_Data_Till_Dec_2024/unzipped-FRD-csv-full'
missing_days_tickers = []

# Cycling through all the appropriate tickers to make sure we have the necessary data
for ticker in tqdm(ticker_list_filtered):

    # Getting the ticker file name and path
    ticker_1min_file = ticker + "_full_1min_UNADJUSTED.csv"
    ticker_1min_path = os.path.join(root_path_orig, ticker_1min_file)

    # Reading in the csv and converting to Datetime column to proper datetime format
    df = pd.read_csv(ticker_1min_path)
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Checking to see if the number of days is not equal to the expected number (86 days from 6/1/20-9/30/20)
    num_days_ticker = df[(df['Datetime'] >= start_date) & (df['Datetime'] < end_date)]['Datetime'].dt.date.nunique()
    if num_days_ticker != num_days_extra:
        print(ticker, "only has", num_days_ticker, "days of 1-min data from 6/1/20-9/30/20.")
        missing_days_tickers.append(ticker)


  0%|▏                                        | 10/2599 [00:03<10:57,  3.94it/s]

ASLE only has 69 days of 1-min data from 6/1/20-9/30/20.


  1%|▎                                        | 17/2599 [00:07<20:14,  2.13it/s]

ASAN only has 1 days of 1-min data from 6/1/20-9/30/20.


  1%|▌                                        | 38/2599 [00:19<14:00,  3.05it/s]

...

 97%|█████████████████████████████████████▊ | 2524/2599 [31:29<00:41,  1.80it/s]

ACI only has 67 days of 1-min data from 6/1/20-9/30/20.


 98%|██████████████████████████████████████▎| 2553/2599 [31:55<00:42,  1.07it/s]

SII only has 66 days of 1-min data from 6/1/20-9/30/20.


100%|███████████████████████████████████████| 2599/2599 [32:35<00:00,  1.33it/s]


In [68]:
# Checking to see how many days of missing data we have:
len(missing_days_tickers)

103

We now see that we still have a substantial amount of missing data, likely because the ticker didn't exist/was still private until 10/1/20, or that it simply didn't have the data. In either case, I've decided to stick with the data since 10/1/20 and perform my analyses from that. It's unfortunate that we'll be missing the extra days since 6/1/20, but it might be for the better anyway since that might introduce existing noise from the COVID-19 crash.

## Obtaining the cleaned 1-day data from the cleaned FRD 1-min dataset
Because I still don't trust the day data from FRD, I'll need to create it myself from the FRD 1-min data (which I've already cleaned anyway).

In [84]:
# Getting the source and destination directories and defining the RTH time window
input_root = "/Volumes/T7/Filtered-Cleaned-Parquet-ML"
output_root = "/Volumes/T7/Filtered-Cleaned-Parquet-1day"
os.makedirs(output_root, exist_ok=True)
rth_start = pd.to_datetime("09:30", format="%H:%M").time()
rth_end = pd.to_datetime("15:59", format="%H:%M").time()

# Cycling through each folder to get the 1-day parquet files
for folder_name in tqdm(os.listdir(input_root)):
    if not folder_name.endswith("_1min_parquet"):
        continue
    
    ticker = folder_name.split("_")[0]
    folder_path = os.path.join(input_root, folder_name)

    try:
        # Reading all intraday parquet files into one DataFrame and adding helper columns
        df = pd.read_parquet(folder_path)
        df['Date'] = df.index.date
        df['Time'] = df.index.time

        # Defining the daily rows that we'll ultimately add to
        daily_rows = []

        # Grouping by trading day
        for date, day_df in df.groupby('Date'):
            rth_df = day_df[(day_df['Time'] >= rth_start) & (day_df['Time'] <= rth_end)]
            pm_df = day_df[day_df['Time'] < rth_start]

            if rth_df.empty:  # Skip if no valid RTH data
                print(date, "for", ticker, "is missing data.")
                continue

            row = {
                "Datetime": pd.Timestamp(date),
                "Open": rth_df.iloc[0]['Open'],
                "High": rth_df['High'].max(),
                "Low": rth_df['Low'].min(),
                "Close": rth_df.iloc[-1]['Close'],
                "Volume": rth_df['Volume'].sum(),
                "PM_High": pm_df['High'].max() if not pm_df.empty else None,
                "PM_Low": pm_df['Low'].min() if not pm_df.empty else None,
                "PM_Volume": pm_df['Volume'].sum() if not pm_df.empty else None,
                "Ticker": ticker
            }

            daily_rows.append(row)

        # Saving daily data
        if daily_rows:
            df_out = pd.DataFrame(daily_rows)
            df_out = df_out.set_index("Datetime")
            output_path = os.path.join(output_root, f"{ticker}_1day.parquet")
            df_out.to_parquet(output_path)

    except Exception as e:
        print(f"Failed processing {folder_name}: {e}")

100%|███████████████████████████████████████| 2599/2599 [49:52<00:00,  1.15s/it]


In [85]:
# We can check to make sure that we still have the same number of files as before
len(os.listdir("/Volumes/T7/Filtered-Cleaned-Parquet-1day"))

2599

In [86]:
# We can also check to make sure the files look appropriate
AAPL_1day_path = '/Volumes/T7/Filtered-Cleaned-Parquet-1day/AAPL_1day.parquet'
pd.read_parquet(AAPL_1day_path)

,Open,High,Low,Close,Volume,PM_High,PM_Low,PM_Volume,Ticker
Datetime,,,,,,,,,
2020-10-01,117.700,117.72,115.83,116.80,92076075.0,118.1000,116.510,1755406.0,AAPL
2020-10-02,112.840,115.37,112.22,113.02,116084401.0,117.0800,112.560,4460967.0,AAPL
2020-10-05,113.920,116.65,113.55,116.54,84421170.0,114.7100,113.300,1690948.0,AAPL
2020-10-06,115.680,116.12,112.25,113.16,132805466.0,116.7900,115.230,1358666.0,AAPL
2020-10-07,114.640,115.55,114.13,115.05,78303638.0,114.7500,113.134,2071097.0,AAPL
...,...,...,...,...,...,...,...,...,...
2024-12-04,242.870,244.11,241.25,243.02,20217543.0,243.6900,242.044,466815.0,AAPL
2024-12-05,243.990,244.54,242.13,243.08,18743262.0,244.2767,242.100,206871.0,AAPL
2024-12-06,242.905,244.63,242.08,242.86,19207219.0,243.2300,242.410,169685.0,AAPL


We now have cleaned day data that we can use for every ticker, and we can now finally start our machine learning process. I will perform my analyses in a separate notebook, using all the cleaned data from my SSD.

## (Update) Adding in PM Highs, Lows, Close, and total Volume up till 9:27am and 9:28am
I thought of this after the fact, but I realized that if I solely want to use PM metrics up till either 9:27am or 9:28am and didn't want to pull out the minute data right away, I'll need pre-calculated metrics for them. The reasoning is simply because I want to potentially trade at the Open, which means that my model will need to run before the Open occurs. Using a PM_threshold of 9:28am will give me one minute for inference, whereas a PM_threshold of 9:27am will give me two minutes. I'm not sure exactly how long I'll need, so I'll get both metrics to be safe.

The PM High, Low, and Volume metrics will be calculated the same way as before, but instead of going to 9:29am, they will only go to 9:27am or 9:28am. The PM_Close at either time will be the Close price of that CS, and this will act as the new "Open" price when calculating metrics such as daily gap ups and such.

In [39]:
# Getting the different paths and defining the PM time cutoffs
minute_root = "/Volumes/T7/Filtered-Cleaned-Parquet-ML"
day_root = "/Volumes/T7/Filtered-Cleaned-Parquet-1day"
pm_cutoff_0927 = pd.to_datetime("09:27", format="%H:%M").time()
pm_cutoff_0928 = pd.to_datetime("09:28", format="%H:%M").time()

# Processing each ticker's 1-day parquet file
for file_name in tqdm(os.listdir(day_root)):
    if not file_name.endswith("_1day.parquet"):
        continue

    ticker = file_name.split("_")[0]
    day_file_path = os.path.join(day_root, file_name)
    minute_folder_path = os.path.join(minute_root, f"{ticker}_1min_parquet")

    try:
        # Loading the 1-day summary df
        day_df = pd.read_parquet(day_file_path)
        day_df.index = pd.to_datetime(day_df.index)

        # Loading all the 1-min data for this ticker
        df_minute = pd.read_parquet(minute_folder_path)
        df_minute.index = pd.to_datetime(df_minute.index)

        # Adding in Date and Time columns for grouping and filtering
        df_minute['Date'] = df_minute.index.date
        df_minute['Time'] = df_minute.index.time

        # Preparing lists for new columns
        pm_high_0927_list = []
        pm_low_0927_list = []
        pm_close_0927_list = []
        pm_volume_0927_list = []

        pm_high_0928_list = []
        pm_low_0928_list = []
        pm_close_0928_list = []
        pm_volume_0928_list = []

        # Grouping by date and calculating metrics
        for date, day_min_df in df_minute.groupby('Date'):
            if pd.Timestamp(date) not in day_df.index:
                # Skip dates not in the 1-day summary
                continue

            # Filtering for PM metrics up to 9:27am
            pm_0927_df = day_min_df[day_min_df['Time'] <= pm_cutoff_0927]
            if not pm_0927_df.empty:
                pm_high_0927_list.append(pm_0927_df['High'].max())
                pm_low_0927_list.append(pm_0927_df['Low'].min())
                pm_close_0927_list.append(pm_0927_df.iloc[-1]['Close'])
                pm_volume_0927_list.append(pm_0927_df['Volume'].sum())
            else:
                pm_high_0927_list.append(None)
                pm_low_0927_list.append(None)
                pm_close_0927_list.append(None)
                pm_volume_0927_list.append(None)
            
            # Filtering for PM metrics up to 9:28am
            pm_0928_df = day_min_df[day_min_df['Time'] <= pm_cutoff_0928]
            if not pm_0928_df.empty:
                pm_high_0928_list.append(pm_0928_df['High'].max())
                pm_low_0928_list.append(pm_0928_df['Low'].min())
                pm_close_0928_list.append(pm_0928_df.iloc[-1]['Close'])
                pm_volume_0928_list.append(pm_0928_df['Volume'].sum())
            else:
                pm_high_0928_list.append(None)
                pm_low_0928_list.append(None)
                pm_close_0928_list.append(None)
                pm_volume_0928_list.append(None)

        # Ensuring the lists align with day_df
        day_df = day_df.loc[pd.to_datetime([pd.Timestamp(d).date() for d in df_minute['Date'].unique()])]

        day_df['PM_High_0927'] = pm_high_0927_list
        day_df['PM_Low_0927'] = pm_low_0927_list
        day_df['PM_Close_0927'] = pm_close_0927_list
        day_df['PM_Volume_0927'] = pm_volume_0927_list

        day_df['PM_High_0928'] = pm_high_0928_list
        day_df['PM_Low_0928'] = pm_low_0928_list
        day_df['PM_Close_0928'] = pm_close_0928_list
        day_df['PM_Volume_0928'] = pm_volume_0928_list

        # Saving updated 1-day parquet file
        day_df.to_parquet(day_file_path)

    except Exception as e:
        print(f"Failed processing {file_name}: {e}")

100%|███████████████████████████████████████| 2599/2599 [37:18<00:00,  1.16it/s]


In [40]:
# We can check to make sure that we still have the same number of files as before
len(os.listdir("/Volumes/T7/Filtered-Cleaned-Parquet-1day"))

2599

In [41]:
# We can also check to make sure the files look appropriate
AAPL_1day_path = '/Volumes/T7/Filtered-Cleaned-Parquet-1day/AAPL_1day.parquet'
pd.read_parquet(AAPL_1day_path)

,Open,High,Low,Close,Volume,PM_High,PM_Low,PM_Volume,Ticker,PM_High_0927,PM_Low_0927,PM_Close_0927,PM_Volume_0927,PM_High_0928,PM_Low_0928,PM_Close_0928,PM_Volume_0928
2020-10-01,117.700,117.72,115.83,116.80,92076075.0,118.1000,116.510,1755406.0,AAPL,118.10,116.510,117.4500,1690772.0,118.1000,116.510,117.6800,1726947.0
2020-10-02,112.840,115.37,112.22,113.02,116084401.0,117.0800,112.560,4460967.0,AAPL,117.08,112.700,113.0000,4319384.0,117.0800,112.560,112.7500,4389769.0
2020-10-05,113.920,116.65,113.55,116.54,84421170.0,114.7100,113.300,1690948.0,AAPL,114.71,113.300,113.9700,1671728.0,114.7100,113.300,113.9200,1685852.0
2020-10-06,115.680,116.12,112.25,113.16,132805466.0,116.7900,115.230,1358666.0,AAPL,116.79,115.230,116.0100,1303941.0,116.7900,115.230,115.8500,1331333.0
2020-10-07,114.640,115.55,114.13,115.05,78303638.0,114.7500,113.134,2071097.0,AAPL,114.75,113.134,114.7000,2032261.0,114.7500,113.134,114.6000,2042679.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-04,242.870,244.11,241.25,243.02,20217543.0,243.6900,242.044,466815.0,AAPL,243.69,242.044,242.8200,457759.0,243.6900,242.044,243.0500,463699.0
2024-12-05,243.990,244.54,242.13,243.08,18743262.0,244.2767,242.100,206871.0,AAPL,244.06,242.100,244.0600,166440.0,244.2767,242.100,243.9400,196704.0
2024-12-06,242.905,244.63,242.08,242.86,19207219.0,243.2300,242.410,169685.0,AAPL,243.20,242.410,242.9799,163273.0,243.2300,242.410,242.9690,168492.0
2024-12-09,241.900,247.24,241.75,246.82,24141189.0,243.0000,241.570,193561.0,AAPL,243.00,241.970,242.1200,166778.0,243.0000,241.570,241.8899,187306.0


With this updated data, we can now use PM prices that more closely reflect what will happen in reality. Furthermore, even though the process is slightly different from before (we're reading in the parquet files rather than creating them outright), it still took less time than before, which gives me more confidence that this Macbook can run more quickly than my previous Mac.